## Creating a Regression Model

In this exercise, you will implement a regression model that uses features of a flight to predict how late or early it will arrive.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "4098m").\
        getOrCreate()

In [3]:
!pip install numpy

     |████████████████████████████████| 15.3 MB 7.1 MB/s eta 0:00:01    |████                            | 1.9 MB 5.2 MB/s eta 0:00:03     |████████████                    | 5.8 MB 5.2 MB/s eta 0:00:02     |██████████████▉                 | 7.1 MB 5.8 MB/s eta 0:00:02


In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating how many minutes late or early the flight arrived.

You will load this data into a DataFrame and display it.

In [5]:
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
  StructField("Late", IntegerType(), False),
])

data = spark.read.csv('../data/flights.csv', schema=flightSchema, header=True)
data.show()

+----------+---------+-------+---------------+-------------+--------+--------+----+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|Late|
+----------+---------+-------+---------------+-------------+--------+--------+----+
|        21|        2|     WN|          10721|        13342|      26|      57|   1|
|        13|        1|     AA|          15016|        12892|      51|      27|   1|
|         5|        5|     FL|          10397|        11433|       9|       4|   0|
|        22|        1|     US|          11278|        14100|      35|      71|   1|
|        23|        4|     WN|          12451|        10693|       9|       5|   0|
|         5|        7|     AA|          11298|        15016|      39|      42|   1|
|         4|        6|     UA|          13930|        14307|      71|      58|   1|
|        10|        3|     9E|          14307|        11433|      68|     140|   1|
|        29|        7|     UA|          14057|        14771|     130|     12

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [6]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 445077  Testing Rows: 192006


### Prepare the Training Data
To train the regression model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **StringIndexer** class to generate a numeric category for each discrete **Carrier** string value, and then use the **VectorAssembler** class to transform the numeric features that would be available for a flight that hasn't yet arrived into a vector, and then rename the **ArrDelay** column to **label** as this is what we're going to try to predict.

*Note: This is a deliberately simple example. In reality you'd likely perform mulitple data preparation steps, and later in this course we'll examine how to encapsulate these steps in to a pipeline. For now, we'll just use the numeric features as they are to dewfine the traaining dataset.*

In [7]:
# Carrier is a string, and we need our features to be numeric - so we'll generate a numeric index for each distinct carrier string, and transform the dataframe to add that as a column
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
numTrain = carrierIndexer.fit(train).transform(train)

# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "CarrierIdx", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(numTrain).select(col("features"), (col("ArrDelay").cast("Int").alias("label")))
training.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,1.0,10.0,103...|  113|
|[1.0,1.0,10.0,103...|    2|
|[1.0,1.0,10.0,105...|   41|
|[1.0,1.0,10.0,107...|   -2|
|[1.0,1.0,10.0,107...|   32|
|[1.0,1.0,10.0,107...|   28|
|[1.0,1.0,10.0,107...|   30|
|[1.0,1.0,10.0,107...|   30|
|[1.0,1.0,10.0,108...|   39|
|[1.0,1.0,10.0,108...|   39|
|[1.0,1.0,10.0,110...|   -2|
|[1.0,1.0,10.0,110...|   76|
|[1.0,1.0,10.0,110...|   82|
|[1.0,1.0,10.0,110...|  -12|
|[1.0,1.0,10.0,111...|   86|
|[1.0,1.0,10.0,111...|   69|
|[1.0,1.0,10.0,111...|   56|
|[1.0,1.0,10.0,111...|  -10|
|[1.0,1.0,10.0,111...|   62|
|[1.0,1.0,10.0,111...|  138|
+--------------------+-----+
only showing top 20 rows



### Train a Regression Model
Next, you need to train a regression model using the training data. To do this, create an instance of the regression algorithm you want to use and use its **fit** method to train a model based on the training DataFrame. In this exercise, you will use a *Linear Regression* algorithm - though you can use the same technique for any of the regression algorithms supported in the spark.ml API.

In [8]:
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
model = lr.fit(training)
print ("Model trained!")

Model trained!


### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **ArrDelay** column to **trueLabel**.

In [9]:
# Transform the test data to add the numeric carrier index
numTest = carrierIndexer.fit(test).transform(test)

# Generate the features vector and label
testing = assembler.transform(numTest).select(col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testing.show()

+--------------------+---------+
|            features|trueLabel|
+--------------------+---------+
|[1.0,1.0,10.0,107...|      -10|
|[1.0,1.0,10.0,108...|       -5|
|[1.0,1.0,10.0,110...|       83|
|[1.0,1.0,10.0,110...|        7|
|[1.0,1.0,10.0,111...|       27|
|[1.0,1.0,10.0,111...|      144|
|[1.0,1.0,10.0,111...|       44|
|[1.0,1.0,10.0,111...|       -3|
|[1.0,1.0,10.0,111...|       54|
|[1.0,1.0,10.0,114...|        0|
|[1.0,1.0,10.0,114...|       26|
|[1.0,1.0,10.0,114...|      -15|
|[1.0,1.0,10.0,114...|        4|
|[1.0,1.0,10.0,114...|       -3|
|[1.0,1.0,10.0,114...|       -1|
|[1.0,1.0,10.0,114...|       94|
|[1.0,1.0,10.0,123...|       28|
|[1.0,1.0,10.0,123...|      -14|
|[1.0,1.0,10.0,124...|        0|
|[1.0,1.0,10.0,124...|       67|
+--------------------+---------+
only showing top 20 rows



### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict arrival delay for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted number of minutes late or early to the actual arrival delay.

In [10]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show()

+--------------------+-------------------+---------+
|            features|         prediction|trueLabel|
+--------------------+-------------------+---------+
|[1.0,1.0,10.0,107...|-0.3331128136469683|      -10|
|[1.0,1.0,10.0,108...| 2.2020961097922926|       -5|
|[1.0,1.0,10.0,110...|  33.99899924497674|       83|
|[1.0,1.0,10.0,110...| 17.771309339685622|        7|
|[1.0,1.0,10.0,111...|  34.75340550446775|       27|
|[1.0,1.0,10.0,111...| 142.54139771642556|      144|
|[1.0,1.0,10.0,111...|  56.20042725507666|       44|
|[1.0,1.0,10.0,111...|  3.301811476424512|       -3|
|[1.0,1.0,10.0,111...| 60.667774850096734|       54|
|[1.0,1.0,10.0,114...| 4.3187136343140295|        0|
|[1.0,1.0,10.0,114...|  5.805069681993495|       26|
|[1.0,1.0,10.0,114...|  9.137139762601768|      -15|
|[1.0,1.0,10.0,114...|0.47007219592140714|        4|
|[1.0,1.0,10.0,114...|  5.172200341304376|       -3|
|[1.0,1.0,10.0,114...|-0.7390733487916386|       -1|
|[1.0,1.0,10.0,114...|   52.8651875085742|    

Looking at the result, the **prediction** column contains the predicted value for the label, and the **trueLabel** column contains the actual known value from the testing data. It looks like there is some variance between the predictions and the actual values (the individual differences are referred to as *residuals*)- later in this course you'll learn how to measure the accuracy of a model.